In [59]:
import glob
import json
import datetime
from dateutil import parser

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML, Image, Video

%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-pastel')

In [60]:
def convert_timeslice_to_buy_sell_curve(df1):
    df1 = df1.values.tolist()
    
    data = {
        "buy_curve": [],
        "sell_curve": [],
        "accepted_buy_volume": None,          # Accepted advance (e.g. block) orders
        "accepted_sell_volume": None,         # Accepted advance (e.g. block) orders
        #"nonaccepted_buy_volume": None,      # Only available before 2016
        #"nonaccepted_sell_volume": None,     # Only available before 2016
        "imported_volume_volume": None        # Can be negative
    }

    is_buy_or_sell_curve = False
    is_buy_curve = True

    last_recorded_volume = 0.0
    last_recorded_price = 0.0
    
    for i, row in enumerate(df1):
        if not is_buy_or_sell_curve:
            if row[0] == "Buy curve":
                is_buy_or_sell_curve = True
                
            elif row[0] == "Bid curve chart data (Volume for accepted blocks buy)":
                data["accepted_buy_volume"] = float(row[1])
            
            elif row[0] == "Bid curve chart data (Volume for accepted blocks sell)":
                data["accepted_sell_volume"] = float(row[1])
            
            #elif row[0] == "Bid curve chart data (Volume for non-accepted blocks buy)":
            #    data["nonaccepted_buy_volume"] = float(row[1])
            #
            #elif row[0] == "Bid curve chart data (Volume for non-accepted blocks sell)":
            #    data["nonaccepted_sell_volume"] = float(row[1])
            
            elif row[0] == "Bid curve chart data (Volume for net flows)":
                data["imported_volume_volume"] = float(row[1])
            
            else:
                continue
        else:
            if row[0] == "Sell curve":
                is_buy_curve = False
                continue

            elif row[0] == "Price value":
                price = float(row[1])
                # Wait for volume to record it

            elif row[0] == "Volume value":
                volume = float(row[1])
                
                # Record setting if significantly changed
                if (abs(volume - last_recorded_volume) > 1000    #MW
                    or abs(price - last_recorded_price) > 1): # EUR/MWh
                    
                    last_recorded_volume = volume
                    last_recorded_price = price

                    if is_buy_curve:
                        data["buy_curve"].append((price, volume))
                    else:
                        data["sell_curve"].append((price, volume))

            elif str(row[0]) == "nan":
                break 

            else:
                print(row)
                raise Exception("Unknown value in row %i: %s" % (i, row[0]))

    data["buy_curve"] = pd.DataFrame(data["buy_curve"], columns=["Price", "Volume"])
    data["sell_curve"] = pd.DataFrame(data["sell_curve"], columns=["Price", "Volume"])
    
    return data


In [61]:
def find_intersection(buy_curve, sell_curve):
    price = None
    volume = None

    i = 0; j = 0
    while i<len(buy_curve) and j<len(sell_curve):

        #print(i, j, buy_curve.iloc[i]["Volume"], sell_curve.iloc[j]["Volume"], buy_curve.iloc[i]["Price"], sell_curve.iloc[j]["Price"])

        if buy_curve.iloc[i]["Volume"] < sell_curve.iloc[j]["Volume"]:
            price = buy_curve.iloc[i]["Price"]
            volume = buy_curve.iloc[i]["Volume"]
            break
        else:
            if buy_curve.iloc[i]["Price"] < sell_curve.iloc[j]["Price"]:
                i += 1
            else:
                j += 1

    return volume, price

In [62]:
buy_and_sell_curves = []

files = glob.glob("data/mcp_data_report_*")
#files = sorted(files, key=lambda file: parser.parse(file[16:26], dayfirst=True)) # Uten data/
files = sorted(files, key=lambda file: parser.parse(file[21:31], dayfirst=True))
#files = files[-2:]

for index, file in enumerate(files):
    progress = "{}/{} ".format(index+1, len(files))
    
    try:
        df = pd.read_excel(file)
        
        for i in range(24):
            date_time_str = df.iloc[:,(i*2):(i*2+2)].columns[1]
            date_time = parser.parse(date_time_str[:19], dayfirst=True)
            data = convert_timeslice_to_buy_sell_curve(df.iloc[:,(i*2):(i*2+2)])
            
            data["buy_curve"]["Volume"] = data["buy_curve"]["Volume"] + data["accepted_buy_volume"] - data["imported_volume_volume"]
            data["sell_curve"]["Volume"] = data["sell_curve"]["Volume"] + data["accepted_sell_volume"]
            
            volume, price = find_intersection(data["buy_curve"], data["sell_curve"])
            
            # New data
            data["volume"] = volume
            data["price"] = price
            data["date_time"] = date_time
            
            buy_and_sell_curves.append(data)
            
    except Exception as e:
        print(e)
        print(progress + file + " FAILED!")
    else:
        print(progress + file)
        #print(buy_and_sell_curves[index*24][0], buy_and_sell_curves[index*24][3])
        #print(buy_and_sell_curves[index*24+1][0], buy_and_sell_curves[index*24+1][3])
    
buy_and_sell_curves = sorted(buy_and_sell_curves, key=lambda item: item["date_time"])
#buy_and_sell_curves[17]

Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
1/1878 data/mcp_data_report_01-07-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
2/1878 data/mcp_data_report_02-07-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
3/1878 data/mcp_data_report_03-07-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
4/1878 data/mcp_data_report_04-07-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
5/1878 data/mcp_data_report_05-07-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
6/1878 data/mcp_data_report_06-07-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
7/1878 data/mcp_data_report_07-07-2014-00_00_

Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
103/1878 data/mcp_data_report_11-10-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
104/1878 data/mcp_data_report_12-10-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
105/1878 data/mcp_data_report_13-10-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
106/1878 data/mcp_data_report_14-10-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
107/1878 data/mcp_data_report_15-10-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
108/1878 data/mcp_data_report_16-10-2014-00_00_00.xls FAILED!
Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf<!DOC'
109/1878 data/mcp_data_report_17-

195/1878 data/mcp_data_report_11-01-2015-00_00_00.xls
196/1878 data/mcp_data_report_12-01-2015-00_00_00.xls
197/1878 data/mcp_data_report_13-01-2015-00_00_00.xls
198/1878 data/mcp_data_report_14-01-2015-00_00_00.xls
199/1878 data/mcp_data_report_15-01-2015-00_00_00.xls
200/1878 data/mcp_data_report_16-01-2015-00_00_00.xls
201/1878 data/mcp_data_report_17-01-2015-00_00_00.xls
202/1878 data/mcp_data_report_18-01-2015-00_00_00.xls
203/1878 data/mcp_data_report_19-01-2015-00_00_00.xls
204/1878 data/mcp_data_report_20-01-2015-00_00_00.xls
205/1878 data/mcp_data_report_21-01-2015-00_00_00.xls
206/1878 data/mcp_data_report_22-01-2015-00_00_00.xls
207/1878 data/mcp_data_report_23-01-2015-00_00_00.xls
208/1878 data/mcp_data_report_24-01-2015-00_00_00.xls
209/1878 data/mcp_data_report_25-01-2015-00_00_00.xls
210/1878 data/mcp_data_report_26-01-2015-00_00_00.xls
211/1878 data/mcp_data_report_27-01-2015-00_00_00.xls
212/1878 data/mcp_data_report_28-01-2015-00_00_00.xls
213/1878 data/mcp_data_repor

346/1878 data/mcp_data_report_11-06-2015-00_00_00.xls
347/1878 data/mcp_data_report_12-06-2015-00_00_00.xls
348/1878 data/mcp_data_report_13-06-2015-00_00_00.xls
349/1878 data/mcp_data_report_14-06-2015-00_00_00.xls
350/1878 data/mcp_data_report_15-06-2015-00_00_00.xls
351/1878 data/mcp_data_report_16-06-2015-00_00_00.xls
352/1878 data/mcp_data_report_17-06-2015-00_00_00.xls
353/1878 data/mcp_data_report_18-06-2015-00_00_00.xls
354/1878 data/mcp_data_report_19-06-2015-00_00_00.xls
355/1878 data/mcp_data_report_20-06-2015-00_00_00.xls
356/1878 data/mcp_data_report_21-06-2015-00_00_00.xls
357/1878 data/mcp_data_report_22-06-2015-00_00_00.xls
358/1878 data/mcp_data_report_23-06-2015-00_00_00.xls
359/1878 data/mcp_data_report_24-06-2015-00_00_00.xls
360/1878 data/mcp_data_report_25-06-2015-00_00_00.xls
361/1878 data/mcp_data_report_26-06-2015-00_00_00.xls
362/1878 data/mcp_data_report_27-06-2015-00_00_00.xls
363/1878 data/mcp_data_report_28-06-2015-00_00_00.xls
364/1878 data/mcp_data_repor

498/1878 data/mcp_data_report_10-11-2015-00_00_00.xls
499/1878 data/mcp_data_report_11-11-2015-00_00_00.xls
500/1878 data/mcp_data_report_12-11-2015-00_00_00.xls
501/1878 data/mcp_data_report_13-11-2015-00_00_00.xls
502/1878 data/mcp_data_report_14-11-2015-00_00_00.xls
503/1878 data/mcp_data_report_15-11-2015-00_00_00.xls
504/1878 data/mcp_data_report_16-11-2015-00_00_00.xls
505/1878 data/mcp_data_report_17-11-2015-00_00_00.xls
506/1878 data/mcp_data_report_18-11-2015-00_00_00.xls
507/1878 data/mcp_data_report_19-11-2015-00_00_00.xls
508/1878 data/mcp_data_report_20-11-2015-00_00_00.xls
509/1878 data/mcp_data_report_21-11-2015-00_00_00.xls
510/1878 data/mcp_data_report_22-11-2015-00_00_00.xls
511/1878 data/mcp_data_report_23-11-2015-00_00_00.xls
512/1878 data/mcp_data_report_24-11-2015-00_00_00.xls
513/1878 data/mcp_data_report_25-11-2015-00_00_00.xls
514/1878 data/mcp_data_report_26-11-2015-00_00_00.xls
515/1878 data/mcp_data_report_27-11-2015-00_00_00.xls
516/1878 data/mcp_data_repor

...

1251/1878 data/mcp_data_report_03-12-2017-00_00_00.xls
1252/1878 data/mcp_data_report_04-12-2017-00_00_00.xls
1253/1878 data/mcp_data_report_05-12-2017-00_00_00.xls
1254/1878 data/mcp_data_report_06-12-2017-00_00_00.xls
1255/1878 data/mcp_data_report_07-12-2017-00_00_00.xls
1256/1878 data/mcp_data_report_08-12-2017-00_00_00.xls
1257/1878 data/mcp_data_report_09-12-2017-00_00_00.xls
1258/1878 data/mcp_data_report_10-12-2017-00_00_00.xls
1259/1878 data/mcp_data_report_11-12-2017-00_00_00.xls
1260/1878 data/mcp_data_report_12-12-2017-00_00_00.xls
1261/1878 data/mcp_data_report_13-12-2017-00_00_00.xls
1262/1878 data/mcp_data_report_14-12-2017-00_00_00.xls
1263/1878 data/mcp_data_report_15-12-2017-00_00_00.xls
1264/1878 data/mcp_data_report_16-12-2017-00_00_00.xls
1265/1878 data/mcp_data_report_17-12-2017-00_00_00.xls
1266/1878 data/mcp_data_report_18-12-2017-00_00_00.xls
1267/1878 data/mcp_data_report_19-12-2017-00_00_00.xls
1268/1878 data/mcp_data_report_20-12-2017-00_00_00.xls
1269/1878 

1399/1878 data/mcp_data_report_30-04-2018-00_00_00.xls
1400/1878 data/mcp_data_report_01-05-2018-00_00_00.xls
1401/1878 data/mcp_data_report_02-05-2018-00_00_00.xls
1402/1878 data/mcp_data_report_03-05-2018-00_00_00.xls
1403/1878 data/mcp_data_report_04-05-2018-00_00_00.xls
1404/1878 data/mcp_data_report_05-05-2018-00_00_00.xls
1405/1878 data/mcp_data_report_06-05-2018-00_00_00.xls
1406/1878 data/mcp_data_report_07-05-2018-00_00_00.xls
1407/1878 data/mcp_data_report_08-05-2018-00_00_00.xls
1408/1878 data/mcp_data_report_09-05-2018-00_00_00.xls
1409/1878 data/mcp_data_report_10-05-2018-00_00_00.xls
1410/1878 data/mcp_data_report_11-05-2018-00_00_00.xls
1411/1878 data/mcp_data_report_12-05-2018-00_00_00.xls
1412/1878 data/mcp_data_report_13-05-2018-00_00_00.xls
1413/1878 data/mcp_data_report_14-05-2018-00_00_00.xls
1414/1878 data/mcp_data_report_15-05-2018-00_00_00.xls
1415/1878 data/mcp_data_report_16-05-2018-00_00_00.xls
1416/1878 data/mcp_data_report_17-05-2018-00_00_00.xls
1417/1878 

1548/1878 data/mcp_data_report_26-09-2018-00_00_00.xls
1549/1878 data/mcp_data_report_27-09-2018-00_00_00.xls
1550/1878 data/mcp_data_report_28-09-2018-00_00_00.xls
1551/1878 data/mcp_data_report_29-09-2018-00_00_00.xls
1552/1878 data/mcp_data_report_30-09-2018-00_00_00.xls
1553/1878 data/mcp_data_report_01-10-2018-00_00_00.xls
1554/1878 data/mcp_data_report_02-10-2018-00_00_00.xls
1555/1878 data/mcp_data_report_03-10-2018-00_00_00.xls
1556/1878 data/mcp_data_report_04-10-2018-00_00_00.xls
1557/1878 data/mcp_data_report_05-10-2018-00_00_00.xls
1558/1878 data/mcp_data_report_06-10-2018-00_00_00.xls
1559/1878 data/mcp_data_report_07-10-2018-00_00_00.xls
1560/1878 data/mcp_data_report_08-10-2018-00_00_00.xls
1561/1878 data/mcp_data_report_09-10-2018-00_00_00.xls
1562/1878 data/mcp_data_report_10-10-2018-00_00_00.xls
1563/1878 data/mcp_data_report_11-10-2018-00_00_00.xls
1564/1878 data/mcp_data_report_12-10-2018-00_00_00.xls
1565/1878 data/mcp_data_report_13-10-2018-00_00_00.xls
1566/1878 

1697/1878 data/mcp_data_report_25-02-2019-00_00_00.xls
1698/1878 data/mcp_data_report_26-02-2019-00_00_00.xls
1699/1878 data/mcp_data_report_27-02-2019-00_00_00.xls
1700/1878 data/mcp_data_report_28-02-2019-00_00_00.xls
1701/1878 data/mcp_data_report_01-03-2019-00_00_00.xls
1702/1878 data/mcp_data_report_02-03-2019-00_00_00.xls
1703/1878 data/mcp_data_report_03-03-2019-00_00_00.xls
1704/1878 data/mcp_data_report_04-03-2019-00_00_00.xls
1705/1878 data/mcp_data_report_05-03-2019-00_00_00.xls
1706/1878 data/mcp_data_report_06-03-2019-00_00_00.xls
1707/1878 data/mcp_data_report_07-03-2019-00_00_00.xls
1708/1878 data/mcp_data_report_08-03-2019-00_00_00.xls
1709/1878 data/mcp_data_report_09-03-2019-00_00_00.xls
1710/1878 data/mcp_data_report_10-03-2019-00_00_00.xls
1711/1878 data/mcp_data_report_11-03-2019-00_00_00.xls
1712/1878 data/mcp_data_report_12-03-2019-00_00_00.xls
1713/1878 data/mcp_data_report_13-03-2019-00_00_00.xls
1714/1878 data/mcp_data_report_14-03-2019-00_00_00.xls
1715/1878 

1845/1878 data/mcp_data_report_23-07-2019-00_00_00.xls
1846/1878 data/mcp_data_report_24-07-2019-00_00_00.xls
1847/1878 data/mcp_data_report_25-07-2019-00_00_00.xls
1848/1878 data/mcp_data_report_26-07-2019-00_00_00.xls
1849/1878 data/mcp_data_report_27-07-2019-00_00_00.xls
1850/1878 data/mcp_data_report_28-07-2019-00_00_00.xls
1851/1878 data/mcp_data_report_29-07-2019-00_00_00.xls
1852/1878 data/mcp_data_report_30-07-2019-00_00_00.xls
1853/1878 data/mcp_data_report_31-07-2019-00_00_00.xls
1854/1878 data/mcp_data_report_01-08-2019-00_00_00.xls
1855/1878 data/mcp_data_report_02-08-2019-00_00_00.xls
1856/1878 data/mcp_data_report_03-08-2019-00_00_00.xls
1857/1878 data/mcp_data_report_04-08-2019-00_00_00.xls
1858/1878 data/mcp_data_report_05-08-2019-00_00_00.xls
1859/1878 data/mcp_data_report_06-08-2019-00_00_00.xls
1860/1878 data/mcp_data_report_07-08-2019-00_00_00.xls
1861/1878 data/mcp_data_report_08-08-2019-00_00_00.xls
1862/1878 data/mcp_data_report_09-08-2019-00_00_00.xls
1863/1878 

In [63]:
def myconverter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()
    if isinstance(o, pd.DataFrame):
        return o.values.tolist()

buy_and_sell_curves_org = buy_and_sell_curves
j = json.dumps(buy_and_sell_curves, default = myconverter)

with open('data.json', 'w') as file:
    file.write(j)